<a href="https://colab.research.google.com/github/cristiandarioortegayubro/BDS/blob/main/modulo.05/bds_evaluacion_002_00.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>


<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20Scikit-learn.png?raw=true">
</p>

 # **<font color="DeepPink">Métricas de evaluación del desempeño: Clustering</font>**

<p align="justify">
En el contexto de clustering, las métricas de evaluación se utilizan para medir qué tan bien se agrupan los datos. Estas métricas permiten evaluar la calidad de los clusters generados por el algoritmo de clustering y seleccionar el número óptimo de clusters, si es necesario.
<br><br>
Algunas de las métricas más comunes utilizadas en clustering son el puntaje de Silueta (<b>Silhouette Score</b>), el índice de Davies-Bouldin (<b>Davies-Bouldin Index</b>), la inercia (<b>Inertia</b>) y el índice de Calinski-Harabasz (<b>Calinski-Harabasz Index</b>).
<br><br>
Es importante destacar que no hay una métrica única y universalmente adecuada para todos los conjuntos de datos y problemas de clustering. La elección de la métrica dependerá de la naturaleza de los datos, la forma en que se agrupan los datos y el objetivo específico que se busca alcanzar con el análisis de clustering. Además, es común utilizar múltiples métricas para tener una evaluación más completa del rendimiento del algoritmo de clustering y la calidad de los clusters generados.


 # **<font color="DeepPink">Carga de las librerías básicas</font>**

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

 # **<font color="DeepPink">Conjunto de datos</font>**

In [ ]:
url = "https://raw.githubusercontent.com/cristiandarioortegayubro/BA/main/Datasets/clientes_mall.csv"

In [ ]:
datos = pd.read_csv(url)

In [ ]:
datos.head()

,CustomerID,Gender,Age,AnnualIncome,SpendingScore
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


<p align="justify">
Este conjuntod de datos contiene información sobre las persona que han visitado un centro comercial. Contiene las siguientes variables: <code>CustomerID</code>, <code>Gender</code>, <code>Age</code>, <code>AnnualIncome</code> y <code>SpendingScore</code> (puntaje asignado por el centro comercial basado en el comportamiento del cliente y la naturaleza del gasto).


In [ ]:
datos = datos.drop(columns=["CustomerID", "Gender"])                            #se eliminan las variable no relevantes
datos

,Age,AnnualIncome,SpendingScore
0,19,15,39
1,21,15,81
2,20,16,6
3,23,16,77
4,31,17,40
...,...,...,...
195,35,120,79
196,45,126,28
197,32,126,74
198,32,137,18


In [ ]:
# Renombramos columnas
datos.rename(columns={"Age": "Edad",
                      "AnnualIncome":"IngresoAnual",
                      "SpendingScore":"ScoreGasto"},inplace=True)

In [ ]:
datos.head()

,Edad,IngresoAnual,ScoreGasto
0,19,15,39
1,21,15,81
2,20,16,6
3,23,16,77
4,31,17,40


 ## **<font color="DeepPink">Escalado de datos</font>**

<p align="justify">
En los modelos que se basan en la distancia, por ejemplo K-means, las variables deben ser escaladas para que cada una contribuya aproximadamente por igual a los cálculos de distancia. Escalar y centrar las variables para que tengan media 0 y desviación estándar 1, asegura que todas las variables tengan el mismo peso cuando se realice el clustering.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
data_scaled = scaler.fit_transform(datos)
data_scaled[:10]

array([[-1.42456879, -1.73899919, -0.43480148],
       [-1.28103541, -1.73899919,  1.19570407],
       [-1.3528021 , -1.70082976, -1.71591298],
       [-1.13750203, -1.70082976,  1.04041783],
       [-0.56336851, -1.66266033, -0.39597992],
       [-1.20926872, -1.66266033,  1.00159627],
       [-0.27630176, -1.62449091, -1.71591298],
       [-1.13750203, -1.62449091,  1.70038436],
       [ 1.80493225, -1.58632148, -1.83237767],
       [-0.6351352 , -1.58632148,  0.84631002]])

In [ ]:
data_scaled = pd.DataFrame(data_scaled, columns = datos.columns)
data_scaled.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
Edad,200.0,-0.0,1.0,-1.50,-0.72,-0.20,0.73,2.24
IngresoAnual,200.0,-0.0,1.0,-1.74,-0.73,0.04,0.67,2.92
ScoreGasto,200.0,-0.0,1.0,-1.91,-0.60,-0.01,0.89,1.89


In [ ]:
data_scaled.head()

,Edad,IngresoAnual,ScoreGasto
0,-1.424569,-1.738999,-0.434801
1,-1.281035,-1.738999,1.195704
2,-1.352802,-1.700830,-1.715913
3,-1.137502,-1.700830,1.040418
4,-0.563369,-1.662660,-0.395980


 # **<font color="DeepPink">K-means</font>**

<p align="justify">
Un cluster hace referencia a un conjunto de datos (utilizando solo vectores
de entrada) agrupados en base a ciertas similitudes. K-means es el algoritmo más popular y simple de aprendizaje automático no supervisado. Se basa en el concepto de distancia e identifica $K$ número de centroides y luego asigna los puntos de datos al cluster más cercano.

<p align="justify">
Existen 2 conceptos fundamentales para comprender el concepto de clustering y su evaluación:
<ul align="justify">
<li><b>Cohesión</b>: el miembro de cada clúster debe ser lo más cercano posible a los otros miembros del mismo clúster. (Minimizar la distancia intra-cluster)
<li><b>Separación</b>: Los clúster deben estar ampliamente separados entre ellos. (Maximizar la distancia inter-cluster)

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics

<p align="justify">
Con la clase <code>KMeans</code> del módulo <code>cluster</code> de <code>Scikit-Learn</code> se pueden entrenar modelos de clustering utilizando el algoritmo K-means. Puede encontrarse una descripción detallada de todos los hiperparámetros en <a href="https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html">sklearn.cluster.KMeans</a>. Sin embargo, entre sus parámetros destacan:
<br><br>
<ul align="justify">
<li><code>n_clusters</code>: determina el número $K$ de clusters que se van a generar.
<br><br>
<li><code>init</code>: estrategia para asignar los centroides iniciales. Por defecto se emplea <code>k-means++</code>, una estrategia que trata de alejar los centroides lo máximo posible facilitando la convergencia. Sin embargo, esta estrategia puede ralentizar el proceso cuando hay muchos datos, si esto ocurre, es mejor utilizar <code>random</code>.
<br><br>
<li><code>n_init</code>: determina el número de veces que se va a repetir el proceso, cada vez con una asignación aleatoria inicial distinta. Es recomendable que este último valor sea alto, entre 10-25.
<br><br>
<li><code>max_iter</code>: número máximo de iteraciones permitidas.
<br><br>
<li><code>random_state</code>: semilla para garantizar la reproducibilidad de los resultados.

In [ ]:
model_kmeans = KMeans(n_clusters = 5, n_init = 25, random_state = 123)
model_kmeans.fit(datos)

KMeans(n_clusters=5, n_init=25, random_state=123)

In [ ]:
prediction = model_kmeans.predict(datos)
prediction[:10]

array([0, 4, 0, 4, 0, 4, 0, 4, 0, 4], dtype=int32)

In [ ]:
centroids = model_kmeans.cluster_centers_
labels = model_kmeans.labels_

In [ ]:
centroids = pd.DataFrame(centroids, columns=datos.columns)
centroids

,Edad,IngresoAnual,ScoreGasto
0,45.217391,26.304348,20.913043
1,32.692308,86.538462,82.128205
2,40.666667,87.750000,17.583333
3,43.088608,55.291139,49.569620
4,25.521739,26.304348,78.565217


In [ ]:
labels

array([0, 4, 0, 4, 0, 4, 0, 4, 0, 4, 0, 4, 0, 4, 0, 4, 0, 4, 0, 4, 0, 4,
       0, 4, 0, 4, 0, 4, 0, 4, 0, 4, 0, 4, 0, 4, 0, 4, 0, 4, 0, 4, 0, 4,
       0, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 2, 1, 3, 1, 2, 1, 2, 1,
       2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 3, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1,
       2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1,
       2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1,
       2, 1], dtype=int32)

In [ ]:
datos['Cluster'] = labels
datos.head()

,Edad,IngresoAnual,ScoreGasto,Cluster
0,19,15,39,0
1,21,15,81,4
2,20,16,6,0
3,23,16,77,4
4,31,17,40,0


In [ ]:
fig = go.Figure([go.Scatter3d(x = datos.Edad,
                              y = datos.IngresoAnual,
                              z = datos.ScoreGasto,
                              mode = "markers",
                              name = "Clusters",
                              marker = dict(color = datos.Cluster)),

                 go.Scatter3d(x = centroids.Edad,
                              y = centroids.IngresoAnual,
                              z = centroids.ScoreGasto,
                              mode = "markers",
                              name = "Centroide",
                              marker_color = "red",
                              marker = dict(size = 12)),
                 ])

fig.update_layout(title = "Edad, Ingresos Anuales y Score de Gasto")

fig.show()

 ## **<font color="DeepPink">Evaluación del modelo</font>**

<p align="justify">
Se puede evaluar la calidad de los modelos de clustering con las siguientes métricas:



### **<font color="DeepPink">Inercia (Inertia)**

<p align="justify">
La inercia mide la suma de las distancias cuadráticas de cada uno de los puntos de datos a sus centroide más cercano. Un valor más bajo de inercia indica clusters más densos y compactos. Su fórmula es:

$$Inertia=\sum_{k=1}^K\sum_{i=1}^{n_k}||d_i-c_k||^2$$


In [ ]:
inertia = model_kmeans.inertia_
inertia

75350.77917248776

### **<font color="DeepPink">Puntuación de Silueta (Silhouette Score)**

<p align="justify">
Esta métrica cuantifica qué tan bien un punto se ajusta a su propio cluster en comparación con los clusters vecinos más cercanos. El valor de silueta varía entre -1 y 1, donde un valor más cercano a 1 indica que el punto está bien asignado a su cluster, mientras que un valor cercano a -1 indica que el punto podría pertenecer a un cluster diferente. Un valor cercano a 0 sugiere que el punto está cerca del límite entre dos clusters. Su fórmula es:

$$s_i = \frac{b_i - a_i}{max(a_i, b_i)}$$

<br><p align="justify">
donde:
<br><ul align="justify">
<li>$a_i$ es la distancia media entre el punto de datos $i$ y todos los demás puntos en el mismo cluster al que pertenece.
<li>$b_i$ es la distancia media entre el punto de datos $i$ y todos los puntos en el cluster más cercano diferente al que pertenece $i$.

<p align="justify">
Para calcular la Silueta Promedio (Average Silhouette), se promedia la puntuación de silueta de todos los puntos de datos en el conjunto de datos:

$$SS = \frac{\sum_{i=1}^n s_i}{n}$$




In [ ]:
ss = metrics.silhouette_score(datos, labels)
ss.round(2)

0.44

Una puntuación de silueta de 0.44 indica que las agrupaciones tienen una separación moderada y que los puntos dentro de cada cluster están relativamente bien agrupados, pero podría haber alguna superposición o ambigüedad en la separación de algunos clusters.

### **<font color="DeepPink">índice de Calinski-Harabasz (Calinski-Harabasz Index)**

<p align="justify">
También conocido como el índice de distancias entre los clusters, es una métrica que relaciona la separación (distancia inter-cluster) y la cohesión (distancia intra-cluster). Un valor más alto del índice de Calinski-Harabasz indica clusters más densos y bien separados. Su fórmula es:

$$CH=\frac{\frac{\sum_{k=1}^K{n_k}||c_k-c||^2}{K-1}}{\frac{\sum_{k=1}^K\sum_{i=1}^{n_k}||d_i-c_k||^2}{N-K}}$$




In [ ]:
metrics.calinski_harabasz_score(datos, labels)

151.23407037709848

 ## **<font color="DeepPink">Número óptimo de clusters</font>**

<p align="justify">
Determinar el número óptimo de clusters es uno de los pasos más importantes a la hora de aplicar métodos de clustering, sobre todo cuando se trata del algoritmo K-means, donde el número se tiene que especificar antes de entrenar el modelo.

 ### **<font color="DeepPink">Método del codo</font>**

<p align="justify">
El método del codo, también conocido como Elbow, sigue una estrategia comúnmente empleada para encontrar el valor óptimo de un hiperparámetro. La idea es probar un rango de valores del hiperparámetro en cuestión (<code>n_clusters</code>), representar gráficamente los resultados obtenidos con cada uno, e identificar aquel punto de la curva (codo) a partir del cual la mejora deja de ser notable.
<br><br>
Una forma sencilla de estimar el número $K$ óptimo de clusters, es aplicar el algoritmo de K-means para un rango de valores de $K$ e identificar aquel valor a partir del cual la reducción en la suma total de distancia intra-cluster (inertia) deja de ser sustancial. A esta estrategia se la conoce como método del codo o elbow method.



In [ ]:
inertia = []
k_range = range(1, 10)

In [ ]:
for k in k_range:
   model_kmeans = KMeans(n_clusters=k,
                         n_init = 'auto',
                         random_state=123).fit(datos)
   inertia.append(model_kmeans.inertia_)

In [ ]:
clusters = pd.DataFrame({'clusters':k_range,
                         'inertia':inertia})

In [ ]:
fig = px.line(clusters,
              x = "clusters",
              y = "inertia",
              markers = True,
              title = "Metodo del codo",
              template = "gridon")
fig.show()

 ### **<font color="DeepPink">Método silhuette</font>**

<p align="justify">
El método de average silhouette considera como número óptimo de clusters aquel que maximiza la media del valor de silueta de todas las observaciones.

In [ ]:
silhouette = []
k_clusters = range(2, 15)

In [ ]:
for k in k_clusters:
    model_kmeans = KMeans(n_clusters   = k,
                          n_init       = 20,
                          random_state = 123)

    cluster_labels = model_kmeans.fit_predict(datos)
    silhouette_avg = metrics.silhouette_score(datos, cluster_labels)
    silhouette.append(silhouette_avg)

In [ ]:
fig = px.line(x = k_clusters,
              y = silhouette,
              markers = True,
              title="Metodo silhouette",
              template="gridon",
              labels = {"x":"clusters", "y":"silhuette score"})
fig.show()

<p align="justify">
El valor medio de las siluetas se maximiza con 6 clusters. Acorde a este criterio, K = 6 es la mejor opción.
<br><br>
Ambos criterios, elbow y silhouette, identifican el valor K = 6 como valor óptimo de clusters.

 # **<font color="DeepPink">Conclusiones</font>**

<p align="justify">
👀 En este colab nosotros:<br><br>
✅ Realizamos el entrenamiento de un algoritmo de clustering. <br>
✅ Calculamos las métricas del modelo, para determinar la calidad de las agrupaciones. <br>
✅ Determinamos el número óptimo de clusters. <br>

<p align="justify">



<br>
<br>
<p align="center"><b>
💗
<font color="DeepPink">
Hemos llegado al final de nuestro colab, a seguir codeando...
</font>
</p>
<br>
<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>

---
